In [147]:
def bfs(blueprint, bag):
    queue = [bag]
    robots = ["ore", "clay", "obsidian", "geode"]
    costs = ["ore", "clay", "obsidian"]
    robots = {robot: {cost: blueprint[f"{robot}_robot_{cost}_cost"] if f"{robot}_robot_{cost}_cost" in blueprint.keys() else 0 for cost in costs} for robot in robots}
    max_state = {}
    width = 20000
    last_minute = -1
    def cost_func(state):
        cost = state["geode"] + state["geode_robot"] * (24 - state["minute"])
        return cost
    while(len(queue)):
        state = queue.pop(0)
        if(state["minute"] > last_minute):
            last_minute = state["minute"]
            queue.sort(key=lambda x: cost_func(x), reverse=True)
            # print(queue, last_minute)
            queue = queue[:width]
        if(state["minute"] > 23):
            if(max_state == {}):
                max_state = state
            elif(state["geode"] > max_state["geode"]):
                max_state = state
            continue
        # Affordability check
        affordable_robots = {}
        for robot in robots:
            affordable = True
            for cost in robots[robot]:
                if(state[cost] < robots[robot][cost]):
                    affordable = False
                    break
            if(affordable):
                affordable_robots[robot] = True
        # Collect phase
        for item in state:
            if("robot" in item):
                state[item.replace("_robot", "")] += state[item]
        state["minute"] += 1
        # Buy phase
        for robot in affordable_robots:
            new_state = state.copy()
            if(robot == "ore"):
                continue
            new_state[f"{robot}_robot"] += 1
            for cost in robots[robot]:
                new_state[cost] -= robots[robot][cost]
            queue.append(new_state)
        queue.append(state)
        # print(queue, "QUEUE")
        # print()
    print(max_state)
    return max_state

In [146]:
import re
with open('input.txt', 'r') as f:
    input = f.read()
input = input.splitlines()
regex = re.compile(r'Blueprint (?P<blueprint_id>\d+): Each ore robot costs (?P<ore_robot_ore_cost>\d+) ore. Each clay robot costs (?P<clay_robot_ore_cost>\d+) ore. Each obsidian robot costs (?P<obsidian_robot_ore_cost>\d+) ore and (?P<obsidian_robot_clay_cost>\d+) clay. Each geode robot costs (?P<geode_robot_ore_cost>\d+) ore and (?P<geode_robot_obsidian_cost>\d+) obsidian.')
input = [regex.match(x).groupdict() for x in input]
input = [dict([a, int(x)] for a, x in b.items()) for b in input]

for blueprint in input:
    bag = {
        "ore_robot": 1,
        "clay_robot": 0,
        "obsidian_robot": 0,
        "geode_robot": 0,
        "geode": 0,
        "ore": 0, 
        "clay": 0,
        "obsidian": 0,
        "minute": 0
    }
    optimal_bag = bfs(blueprint, bag.copy())


{'ore_robot': 1, 'clay_robot': 6, 'obsidian_robot': 2, 'geode_robot': 2, 'geode': 9, 'ore': 2, 'clay': 47, 'obsidian': 8, 'minute': 24}
{'ore_robot': 1, 'clay_robot': 4, 'obsidian_robot': 2, 'geode_robot': 1, 'geode': 4, 'ore': 3, 'clay': 31, 'obsidian': 10, 'minute': 24}
